<h1>Evaluation of different Stable Distribution Fitting Methods</h1>

This notebook evaluates various methods for fitting stable distributions to data, including the Quantile Method, Log-Moments Method, Tail-Regression Method and Maximum Likelihood implemented in R. Synthetic data from a known stable distribution will be generated to assess the accuracy of each fitting method. Specifically, 100 values within the rainge of α (0 < α ≤ 2) will be tested to compare the estimated parameters of each method against the true values.

In [7]:
import numpy as np
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri, numpy2ri, default_converter
import rpy2.robjects as ro
import pandas as pd

ro.r('.libPaths("/Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/library")')

In [2]:
# activate conversion between R and pandas/numpy
converter = default_converter + numpy2ri.converter + pandas2ri.converter
robjects.conversion.set_conversion(converter)


# load R script
robjects.r['source'](r'/Users/janikwahrheit/Library/CloudStorage/OneDrive-Persönlich/01_Studium/01_Bachelor/Bachelorarbeit/Code/optim/fit_stable.R')

In [ ]:
# call function 
fit_stable_plot = robjects.globalenv['fit_stable_plot']

In [8]:
n_samples = 1000

alphas = np.linspace(0.1, 2.0, 100)

beta = 0.0
gamma = 1.0
delta = 0.0